In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd

import time

from datetime import datetime
from dateutil import relativedelta

XPATH = By.XPATH

url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=12&country=Russian&orderType=Desc&sortType=ByYieldToClient'
# url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=2400&country=Russian&orderType=Desc&sortType=ByYieldToClient'


In [2]:
class Obligations_parser:
    def __init__(self, driver : webdriver.Firefox, url : str, existing_df : pd.DataFrame = pd.DataFrame()):
        self.driver = driver
        self.driver.maximize_window()
        self.prev_window_switch = 0
        self.previous_url = url

        self.driver.get(url)
        self.obligations_data = existing_df.copy()


    def next_layer(self, add_value : int = 12):
        start = self.previous_url[self.previous_url.index('start') : self.previous_url.index('&end')]
        start = int(start[start.index('=') + 1 :]) + add_value

        end = self.previous_url[ self.previous_url.index('end') : self.previous_url.index('&country')]
        end = int(end[end.index('=') + 1 :]) + add_value


        self.previous_url = f"{self.previous_url[ : self.previous_url.index('start') + 6]}{start}&end={end}&{self.previous_url[self.previous_url.index('country') :]}"

        self.driver.switch_to.window(self.driver.window_handles[0])
        self.driver.get(self.previous_url)

        self.elements_iterator()

            
    def grab_paper_info(self, paper_url : str = ''):
        if paper_url:
            self.driver.execute_script("window.open('');")
            self.driver.switch_to.window(self.driver.window_handles[-1])
            self.driver.get(paper_url)
        
        else:
            self.driver.get(paper_url)

        paper_data = {}

        try:
            paper_data.update({'Название' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__showName_iw6qC"]').text]})
            
            paper_data.update({'Код' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__ticker_j7fZW"]').text]})

            cost = self.driver.find_element(XPATH, '//div[@class = "SecurityInvitingScreen__price_FSP8P"]').text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1]
            paper_data.update({'Цена' : [float(cost) if cost else -1]})
            
            paper_data.update({'Рейтинг' : [self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panel_itBzT SecurityHeader__desktop_dL7RD"]/div[@class = "SecurityHeader__panelText_KDJdO"]').text]})

            paper_datas = self.driver.find_elements(XPATH, '//td[@class = "Table-module__cell_RJ0qz"]')


            for counter in range(1, len(paper_datas) + 1, 2):
                # print(paper_datas[counter - 1].text)
                match paper_datas[counter - 1].text:
                    case 'Дата колл-опциона':
                        print('Облигация с колл-опционом')
                        break

                    case 'Дата погашения облигации':
                        expire_date = datetime.strptime(paper_datas[counter].text, '%d.%m.%Y')
                        monthes_delta = relativedelta.relativedelta(expire_date, datetime.now())
                        monthes_delta = monthes_delta.years * 12 + monthes_delta.months

                        paper_data.update({'Срок(месяца)' : [monthes_delta]})


                    case "Величина купона":
                        paper_coupon = float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -2])
                        paper_coupon -= paper_coupon * 0.13     # Вычитаем налог 13% на купоны

                        paper_data.update({'Величина купона(-13%)' : [round(paper_coupon, 2)]})
                    
                    case "Номинал":
                        paper_data.update({'Номинал' : [float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1])]})
                    
                    case "Количество выплат в год":
                        if 'Величина купона(-13%)' not in paper_data:
                            paper_data.update({'Величина купона(-13%)' : [0]})
                            
                        paper_data.update({'Выплат в год' : [int(paper_datas[counter].text)]})
                        paper_data.update({'Доходность(год)' : [round(paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0], 2)]})
                        # paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // 2 * paper_data['Доходность(год)'][0]]})
                        # paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})

                        paper_data.update({'Доходность(все время)' : [round(paper_data['Срок(месяца)'][0] / paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0], 2)]})
                        paper_data.update({'Доходность(% в год)' : [round(paper_data['Доходность(год)'][0] / paper_data['Цена'][0] * 100, 2)]})

                    
                    case 'Амортизация':
                        paper_data.update({'Амортизация' : [True] if paper_datas[counter] == 'Да' else [False]})
                    

            print(paper_data)

            paper_data.update({'Ссылка' : [paper_url]})

            

            # print(pd.DataFrame(paper_data))
            # print(pd.DataFrame.from_dict(paper_data), )

            

            self.obligations_data = pd.concat([self.obligations_data, pd.DataFrame(paper_data)])

            
            self.driver.close()

        except Exception:
            print('Тулево словили ошибулю')
            self.driver.close()
            time.sleep(10)
            self.grab_paper_info()


    def elements_iterator(self):

        # link_buttons_class = 'Link-module__link_yQVl1 Link-module__link_theme_default_mkRhf'

        # Использую тр и тд, поскольку на сайте в строках тр 5 ссылок с нужным адресом, потому просто по классу ссылки не выйдет найти одно совпадение - придется потом чистить список. Приходится потихоьнку находить элементы.
        tr_link_buttons_class = 'Table-module__row_Qlwsh Table-module__row_clickable_FeO1O'
        # td_link_buttons_class = 'Table-module__cell_RJ0qz'

        papers_rows = self.driver.find_elements(XPATH, f'//tr[@class = "{tr_link_buttons_class}"]')
        
        for paper in papers_rows:
            self.driver.switch_to.window(self.driver.window_handles[-1])

            paper_link = paper.find_element(XPATH, 'td').find_element(XPATH, 'a').get_attribute('href')

            
            self.grab_paper_info(paper_link)
            print(len(self.obligations_data), '\n')

        if len(papers_rows):
            self.next_layer()
        
        else:
            return(self.obligations_data)

In [42]:
import plotly.express as px

def plot_papers(df: pd.DataFrame):
    # if 'Доходность(все время)' in df.columns:
    #     target_column_name = 'Доходность(все время)'

    # elif 'Доходность(год)' in df.columns:
    #     target_column_name = 'Доходность(год)'

    # else:
    #     target_column_name = None

    target_column_name = 'Доходность(% в год)'

    # Используйте Plotly Express для создания интерактивного графика
    fig = px.scatter(df, y=target_column_name, x='Цена', color='Код', size_max=10, hover_data=['Код', "Название", "Цена", "Доходность(% в год)", "Срок(месяца)"])

    # Настройки осей и заголовка
    fig.update_xaxes(title_text='Цена')
    fig.update_yaxes(title_text=target_column_name)
    fig.update_layout(title='Зависимость цены от доходности для бумаг')

    # Отображаем график
    fig.show()

In [17]:
# a = webdriver.Chrome()
driver = webdriver.Firefox()

---

In [36]:
def change_url(url, start_val, end_val):
    return(f"{url[ : url.index('start') + 6]}{start_val}&end={end_val}&{url[url.index('country') :]}")

In [37]:
def df_cutter(df, resize_to_n_rows : int):
    return(pd.DataFrame(df.iloc[ : resize_to_n_rows]))

In [38]:
loaded_df = pd.read_csv('Obligations_data_parsed_833_09.10.2023.csv').drop('Unnamed: 0', axis = 1).fillna(0)
# print(loaded_df['Код'].size // 12 * 12)

loaded_df = df_cutter(loaded_df, loaded_df['Код'].size // 12 * 12)

loaded_df = loaded_df[loaded_df['Цена'] > 0]
loaded_df = loaded_df[loaded_df["Цена"] <= 20000]
loaded_df = loaded_df[loaded_df['Доходность(% в год)'] >= 12 ]

In [29]:
driver.switch_to.window(driver.window_handles[0])
parser_object = Obligations_parser(driver=driver, url=change_url(url, loaded_df['Код'].size, loaded_df['Код'].size + 12), existing_df=loaded_df)

---

In [18]:
parser_object = Obligations_parser(driver, url)

In [30]:
parser_object.elements_iterator()

{'Название': ['Европлан выпуск 6'], 'Код': ['RU000A106F40'], 'Цена': [950.0], 'Рейтинг': ['Высокий'], 'Срок(месяца)': [20], 'Величина купона(-13%)': [22.18], 'Номинал': [1000.0], 'Выплат в год': [4], 'Доходность(год)': [88.72], 'Доходность(все время)': [110.9], 'Доходность(% в год)': [9.34]}
277 

{'Название': ['Медси выпуск 001Р-02'], 'Код': ['RU000A105YB1'], 'Цена': [929.7], 'Рейтинг': ['16.03.2026'], 'Срок(месяца)': [172], 'Величина купона(-13%)': [21.75], 'Номинал': [1000.0], 'Выплат в год': [4], 'Доходность(год)': [87.0], 'Доходность(все время)': [935.25], 'Доходность(% в год)': [9.36]}
278 

{'Название': ['СУЭК-Финанс выпуск 3'], 'Код': ['RU000A100VG7'], 'Цена': [950.0], 'Рейтинг': ['26.03.2024'], 'Срок(месяца)': [71], 'Величина купона(-13%)': [8.61], 'Номинал': [1000.0], 'Выплат в год': [2], 'Доходность(год)': [17.22], 'Доходность(все время)': [305.65], 'Доходность(% в год)': [1.81]}
279 

{'Название': ['Новотранс выпуск 3'], 'Код': ['RU000A105CM4'], 'Цена': [945.1], 'Рейтинг': 

In [27]:
df = parser_object.obligations_data.reset_index().drop('index', axis = 1)

In [28]:
df.to_csv(f'Obligations_data_parsed_{df["Код"].size}_{datetime.strftime(datetime.now(), "%d.%m.%Y")}.csv')

In [31]:
plot_papers(df)

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Название', 'Код', 'Цена', 'Рейтинг', 'Срок(месяца)', 'Величина купона(-13%)', 'Номинал', 'Выплат в год', 'Доходность(год)', 'Доходность(все время)', 'Доходность(% в год)', 'Ссылка'] but received: Доходность(%)

In [43]:
plot_papers(loaded_df)

In [110]:
df[df['Номинал'] <= 15000].sort_values(by=['Доходность(% в год)'], ascending=False)

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Доходность(% в год),Ссылка
763,АПРИ Флай Плэнинг,RU000A105DS9,36.0,1013.5,Средний,52.03,1000.0,4.0,208.12,468.27,20.53,https://www.tinkoff.ru/invest/bonds/RU000A105DS9/
50,Завод КЭС выпуск 4,RU000A106QS9,57.0,927.1,Низкий,42.11,1000.0,4.0,168.44,600.07,18.17,https://www.tinkoff.ru/invest/bonds/RU000A106QS9/
16,АПРИ Флай Плэнинг выпуск 3,RU000A106WZ2,47.0,1022.2,Средний,45.50,1000.0,4.0,182.00,534.62,17.80,https://www.tinkoff.ru/invest/bonds/RU000A106WZ2/
47,МФК ВЭББАНКИР выпуск 4,RU000A106Y88,41.0,1002.5,Средний,14.27,1000.0,12.0,171.24,48.76,17.08,https://www.tinkoff.ru/invest/bonds/RU000A106Y88/
818,Мосгорломбард,RU000A104XJ9,38.0,1129.1,Низкий,47.68,1000.0,4.0,190.72,452.96,16.89,https://www.tinkoff.ru/invest/bonds/RU000A104XJ9/
...,...,...,...,...,...,...,...,...,...,...,...,...
826,Россети 001P выпуск 10,RU000A1065A3,78.0,-1.0,Средний,23.49,1000.0,4.0,93.96,458.05,-9396.00,https://www.tinkoff.ru/invest/bonds/RU000A1065A3/
830,Группа Черкизово,RU000A105M67,49.0,-1.0,Средний,24.45,1000.0,4.0,97.80,299.51,-9780.00,https://www.tinkoff.ru/invest/bonds/RU000A105M67/
827,РЖД,RU000A104SY8,42.0,-1.0,Высокий,51.42,1000.0,2.0,102.84,1079.82,-10284.00,https://www.tinkoff.ru/invest/bonds/RU000A104SY8/
828,ППК РЭО выпуск 1,RU000A105RW1,110.0,-1.0,Средний,32.19,1000.0,4.0,128.76,885.22,-12876.00,https://www.tinkoff.ru/invest/bonds/RU000A105RW1/


In [40]:
df[df['Номинал'] <= 15000].sort_values(['Величина купона(-13%)'], ascending=False)

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Доходность(% в год),Ссылка
379,ПЮДМ БО-П2,RU000A1020K7,21,10070.000000,Средний,92.92,10000.00,12.0,1115.04,1951.32,11.15,https://www.tinkoff.ru/invest/bonds/RU000A1020K7/
578,Нафтатранс Плюс выпуск 1,RU000A100303,3,10007.000000,Средний,84.30,10000.00,12.0,1011.60,252.90,10.12,https://www.tinkoff.ru/invest/bonds/RU000A100303/
643,ВЭБ.РФ ПБО-001Р-К400,RU000A104HT1,16,998.800000,Высокий,80.04,1000.00,1.0,80.04,15367.68,8.00,https://www.tinkoff.ru/invest/bonds/RU000A104HT1/
803,Селигдар Золотые облигации,RU000A1062M5,53,5684.582904,Высокий,70.21,5885.88,4.0,280.84,11163.39,4.77,https://www.tinkoff.ru/invest/bonds/RU000A1062M5/
353,РСХБ 08Т1,RU000A0JWV63,35,1028.500000,Средний,61.77,1000.00,2.0,123.54,12971.70,12.35,https://www.tinkoff.ru/invest/bonds/RU000A0JWV63/
...,...,...,...,...,...,...,...,...,...,...,...,...
261,Верхнебаканский цементный завод 001P выпуск 1,RU000A0ZYLK7,50,974.400000,Средний,1.57,1000.00,4.0,6.28,235.50,0.63,https://www.tinkoff.ru/invest/bonds/RU000A0ZYLK7/
344,МСБ-Лизинг 002P выпуск 2,RU000A100DC4,6,166.783320,Средний,1.48,166.80,12.0,17.76,8.88,10.65,https://www.tinkoff.ru/invest/bonds/RU000A100DC4/
494,ТД Мясничий выпуск 3,RU000A1006B5,4,145.449500,Средний,1.39,145.00,12.0,16.68,5.56,11.50,https://www.tinkoff.ru/invest/bonds/RU000A1006B5/
696,Ямало-Ненецкий АО выпуск 2,RU000A0JX0Z8,2,49.900000,Высокий,1.04,50.00,4.0,4.16,6.24,8.32,https://www.tinkoff.ru/invest/bonds/RU000A0JX0Z8/
